In [3]:
import numpy as np
from numpy.linalg import norm

# Updated user ratings
alice = np.array([5, 3, 4, 4, 0])  # Alice has not rated the fifth item
user1 = np.array([3, 1, 2, 3, 3])
user2 = np.array([4, 3, 4, 3, 5])
user3 = np.array([3, 3, 1, 5, 4])
user4 = np.array([1, 5, 5, 2, 1])

# Compile all ratings into a matrix
ratings = np.array([alice, user1, user2, user3, user4])

def cosine_similarity(u1, u2):
    """Computes cosine similarity between two vectors."""
    return np.dot(u1, u2) / (norm(u1) * norm(u2))

# Compute the similarity matrix
num_users = ratings.shape[0]
similarity_matrix = np.zeros((num_users, num_users))

for i in range(num_users):
    for j in range(num_users):
        if i != j:
            similarity_matrix[i, j] = cosine_similarity(ratings[i], ratings[j])
        else:
            similarity_matrix[i, j] = 0  # exclude self-similarity

print("Similarity Matrix:\n", similarity_matrix)

Similarity Matrix:
 [[0.         0.82686887 0.81016272 0.76277007 0.78954203]
 [0.82686887 0.         0.95938348 0.9356927  0.63781505]
 [0.81016272 0.95938348 0.         0.89442719 0.77151675]
 [0.76277007 0.9356927  0.89442719 0.         0.63831064]
 [0.78954203 0.63781505 0.77151675 0.63831064 0.        ]]


In [4]:
def predict_rating(target_user_idx, item_idx, k=3):
    """Predicts a rating for a specific user and item based on their neighbors."""
    # Get similarities for the target user
    similarities = similarity_matrix[target_user_idx]
    print(similarities)
    # Get the top k neighbors excluding the target user
    neighbor_indices = similarities.argsort()[-k:][::-1]
    print(neighbor_indices)
    
    # Compute the predicted rating based on weighted average
    #numerator = sum(similarities[i] * ratings[i, item_idx] for i in neighbor_indices if ratings[i, item_idx] != 0)
    #print(numerator)
    
    #Calculate the numerator explicitly
    numerator = 0
    for idx in neighbor_indices:
        neighbor_similarity = similarities[idx]
        neighbor_rating = ratings[idx, item_idx]
        weighted_rating = neighbor_similarity * neighbor_rating
        numerator += weighted_rating
        print(f"Neighbor {idx}: Similarity={neighbor_similarity}, Rating={neighbor_rating}, Weighted Contribution={weighted_rating}")
    
    print(numerator)
    denominator = sum(abs(similarities[i]) for i in neighbor_indices if ratings[i, item_idx] != 0)
    print(denominator)
    if denominator == 0:
        return np.mean(ratings[target_user_idx][ratings[target_user_idx] != 0])  # Return average if no similar neighbors
    return numerator / denominator

# Predict a rating for Alice (index 0) on the fifth item (index 4)
alice_idx = 0
item_idx = 4  # Fifth item

predicted_rating = predict_rating(alice_idx, item_idx, k=3)
print(f"Predicted Rating for Alice on Item {item_idx}: {predicted_rating}")

[0.         0.82686887 0.81016272 0.76277007 0.78954203]
[1 2 4]
Neighbor 1: Similarity=0.8268688657895646, Rating=3, Weighted Contribution=2.480606597368694
Neighbor 2: Similarity=0.8101627221513196, Rating=5, Weighted Contribution=4.050813610756598
Neighbor 4: Similarity=0.7895420339517227, Rating=1, Weighted Contribution=0.7895420339517227
7.320962242077014
2.426573621892607
Predicted Rating for Alice on Item 4: 3.016995724352689


In [7]:
import numpy as np
from numpy.linalg import norm

# Updated user ratings
alice = np.array([5, 3, 4, 4, 0])  # Alice has not rated the fifth item
user1 = np.array([3, 1, 2, 3, 3])
user2 = np.array([4, 3, 4, 3, 5])
user3 = np.array([3, 3, 1, 5, 4])
user4 = np.array([1, 5, 5, 2, 1])

# Compile all ratings into a matrix
ratings = np.array([alice, user1, user2, user3, user4]).T  # Transpose to get item-based similarity

def cosine_similarity(u1, u2):
    """Computes cosine similarity between two vectors."""
    return np.dot(u1, u2) / (norm(u1) * norm(u2))

# Compute the item similarity matrix
num_items = ratings.shape[0]
item_similarity_matrix = np.zeros((num_items, num_items))

for i in range(num_items):
    for j in range(num_items):
        if i != j:
            item_similarity_matrix[i, j] = cosine_similarity(ratings[i], ratings[j])
        else:
            item_similarity_matrix[i, j] = 0  # exclude self-similarity

print("Item Similarity Matrix:\n", item_similarity_matrix)

Item Similarity Matrix:
 [[0.         0.78025959 0.81978229 0.94337007 0.7592566 ]
 [0.78025959 0.         0.94201969 0.84798442 0.67320142]
 [0.81978229 0.94201969 0.         0.78402509 0.62242512]
 [0.94337007 0.84798442 0.78402509 0.         0.81152606]
 [0.7592566  0.67320142 0.62242512 0.81152606 0.        ]]


In [8]:
def predict_item_based_rating(target_user_idx, target_item_idx, k=3):
    """Predicts a rating for a specific user and item based on similar items."""
    # Get similarities for the target item
    item_similarities = item_similarity_matrix[target_item_idx]
    print(item_similarities)

    # Get the top k similar items excluding the target item
    similar_item_indices = item_similarities.argsort()[-k:][::-1]
    print(similar_item_indices)

    # Compute the weighted sum numerator explicitly
    numerator = 0
    for idx in similar_item_indices:
        if ratings[idx, target_user_idx] != 0:  # Ensure the user rated this item
            item_similarity = item_similarities[idx]
            item_rating = ratings[idx, target_user_idx]
            weighted_rating = item_similarity * item_rating
            numerator += weighted_rating
            print(f"Similar Item {idx}: Similarity={item_similarity}, Rating={item_rating}, Weighted Contribution={weighted_rating}")
    print(numerator)
    # Compute the denominator as the sum of absolute similarities
    denominator = sum(abs(item_similarities[i]) for i in similar_item_indices if ratings[i, target_user_idx] != 0)
    print(denominator)

    if denominator == 0:
        return np.mean(ratings[:, target_user_idx][ratings[:, target_user_idx] != 0])  # Return average if no similar items
    return numerator / denominator

# Predict a rating for Alice (index 0) on the fifth item (index 4)
alice_idx = 0  # First user (Alice)
item_idx = 4  # Fifth item

predicted_rating = predict_item_based_rating(alice_idx, item_idx, k=3)
print(f"Predicted Rating for Alice on Item {item_idx}: {predicted_rating}")

[0.7592566  0.67320142 0.62242512 0.81152606 0.        ]
[3 0 1]
Similar Item 3: Similarity=0.8115260568382767, Rating=4, Weighted Contribution=3.246104227353107
Similar Item 0: Similarity=0.7592566023652966, Rating=5, Weighted Contribution=3.7962830118264828
Similar Item 1: Similarity=0.6732014170987178, Rating=3, Weighted Contribution=2.0196042512961534
9.061991490475743
2.2439840763022914
Predicted Rating for Alice on Item 4: 4.038349285173352
